# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader
from utils.logging_utils import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10, 42, 4] # don't change!
#STRATEGIES = ['freeze', 'shuffle', 'homogeneous', 'heterogeneous', 'max_k_loss', 'min_k_loss'] # can be changed
STRATEGIES = ['max_k_loss']
#['weighted_random_sampling']#, 'max_k_loss']#, 'heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']
UPDATE_STRATEGIES = ['on_test_acc_reduction', 'every_epoch', 'every_iteration']
update_strategy = UPDATE_STRATEGIES[0] # current implementation: choose one during testing

In [4]:
### Training
def train(epoch, optimizer, criterion_fn, seed, dataloader, strategy, device, update_strategy):
    criterion = criterion_fn()
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    update_every_iteration = True if update_strategy == 'every_iteration' else False
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(strategy, \
                                                  random_state=seed, use_train=True, \
                                                  criterion=criterion_fn(reduction='none'),\
                                                  device=device, num_iterations=500,
                                                  update_every_iteration=update_every_iteration)): # 500: 50,000 images / 100 batch size
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('freeze', \
                                                                               random_state=seed, use_train=True)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, criterion_fn, seed, dataloader, strategy, device, last_acc, update_strategy):
    criterion = criterion_fn()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('freeze', \
                                                                               random_state=seed, use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if update_strategy == 'on_test_acc_reduction':
        if test_acc < last_acc:
            if strategy in ['max_k_loss', 'min_k_loss', 'weighted_random_sampling']:
                    dataloader.initialize_weights(criterion_fn(reduction='none'), device, seed=seed, 
                                                  dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, 0))
            if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
                dataloader.initialize_weights_per_class(criterion_fn(reduction='none'), device, seed=seed)
    if test_acc > best_acc:
        best_acc = test_acc
        net.save(best_acc, epoch, seed, strategy)
    return test_acc, test_loss

In [5]:
resume = False
given_date = '20191113' #only needed if resumed from checkpoint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 100 # number of epochs the model gets trained
learning_rate = 0.01
momentum = 0.9
batch_size = 100
weight_decay = 5e-4

print("Begin training.")
start = timeit.default_timer()
#Logging header
length_table = 90
log_separating_line(length_table)
log_header_line("Seeds: {}".format(SEEDS), length_table)
log_header_line("Strategies: {}".format(STRATEGIES), length_table)
log_header_line("-> Resulting number of iterations: {}".format(len(SEEDS) * len(STRATEGIES)), length_table)
log_header_line("Number of iterations: {}".format(num_epochs), length_table)
log_header_line("Learning rate: {}".format(learning_rate), length_table)
log_header_line("Resuming from checkpoint: {}".format(True if resume else False), length_table)
log_separating_line(length_table)

rows = []
for seed in SEEDS:
    for strategy in STRATEGIES:
        np.random.seed(seed)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss # no function!
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        dataloader = DataLoader(batch_size=batch_size)
        dataloader.download_cifar()
        dataloader.set_model(net)
        
        if strategy in ['max_k_loss', 'min_k_loss', 'weighted_random_sampling']:
            dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
                                          dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, 0))
        if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
            dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)
        last_acc = 0
            
        print()
        log_separating_line(length_table)
        log_position_header(seed, strategy, length_table)
        log_separating_line(length_table)

        if resume:
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/{}_ckpt_{}.pth'.format(given_date, strategy, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if update_strategy == 'every_epoch':
                if strategy in ['max_k_loss', 'min_k_loss', 'weighted_random_sampling']:
                    dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
                                                  dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, epoch))
                if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
                    dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)

            train_acc, train_loss = train(epoch, optimizer, criterion, seed, dataloader, \
                                          strategy, device, update_strategy)
            test_acc, test_loss = test(epoch, test_acc, criterion, seed, dataloader, strategy, device, last_acc, update_strategy)
            log_position_line(epoch + 1, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'strategy': strategy,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'strategy': strategy,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
        log_separating_line(length_table)
            
stop = timeit.default_timer()
time_needed = stop - start
hrs = int(time_needed / 3600)
mins = int((time_needed / 60) % 60)
secs = int(time_needed % 60)
print()
print("Finished training. Time needed: {} hrs {} mins {} secs".format(hrs, mins, secs))

logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)

Begin training.
+----------------------------------------------------------------------------------------+
| Seeds: [10, 42, 4]                                                                     |
| Strategies: ['weighted_random_sampling']                                               |
| -> Resulting number of iterations: 3                                                   |
| Number of iterations: 100                                                              |
| Learning rate: 0.01                                                                    |
| Resuming from checkpoint: False                                                        |
+----------------------------------------------------------------------------------------+
Files already downloaded and verified
Files already downloaded and verified

+----------------------------------------------------------------------------------------+
| Seed: 10      Strategy: weighted_random_sampling                                      

| [078/100]:    073.3           62.53           0.01069      0.01801                     |
| [079/100]:    066.2           57.54           0.01504      0.02301                     |
| [080/100]:    75.54           65.42           0.00929      0.01611                     |
| [081/100]:    69.78           058.0           0.01154      0.01981                     |
| [082/100]:    73.53           62.43           0.01056      0.01815                     |
| [083/100]:    74.61           62.15           0.00996      0.01837                     |
| [084/100]:    75.97           63.93           00.0095      0.01665                     |
| [085/100]:    73.29           60.98           0.01071      0.01924                     |
| [086/100]:    73.33           62.28           0.00953      0.01719                     |
| [087/100]:    69.66           58.45           0.01235      000.021                     |
| [088/100]:    75.52           63.65           0.00919      0.01676                     |

| [062/100]:    75.67           64.95           0.00865      0.01502                     |
| [063/100]:    67.28           58.18           00.0143      00.0216                     |
| [064/100]:    78.05           067.7           0.00891      0.01557                     |
| [065/100]:    74.24           63.28           00.0104      00.0165                     |
| [066/100]:    76.26           66.34           0.00909      00.0158                     |
| [067/100]:    073.8           63.01           0.01147      0.01847                     |
| [068/100]:    78.51           66.12           0.00789      00.0154                     |
| [069/100]:    070.2           060.2           0.01256      0.01962                     |
| [070/100]:    75.64           65.67           0.01057      0.01785                     |
| [071/100]:    73.48           62.18           0.01254      0.02017                     |
| [072/100]:    73.68           63.24           0.01211      0.01988                     |

| [046/100]:    68.06           60.32           0.01194      0.01693                     |
| [047/100]:    67.11           57.29           00.0121      0.01892                     |
| [048/100]:    69.84           61.43           0.01156      0.01686                     |
| [049/100]:    65.51           57.23           0.01342      0.02012                     |
| [050/100]:    73.41           64.19           0.00962      0.01516                     |
| [051/100]:    67.02           57.65           0.01238      0.01899                     |
| [052/100]:    71.06           62.19           0.01094      0.01655                     |
| [053/100]:    69.77           060.2           0.01148      0.01849                     |
| [054/100]:    67.88           58.35           0.01297      0.01969                     |
| [055/100]:    74.57           64.44           0.00894      0.01495                     |
| [056/100]:    71.28           60.72           0.00996      0.01673                     |